In [ ]:
# Install the transformer library
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing libraries and setting random seeds
import numpy as np
import pandas as pd
import random as python_random
import json
import argparse
import tensorflow as tf
import re

from sklearn.model_selection import train_test_split
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import TextVectorization
from sklearn.preprocessing import LabelBinarizer
from transformers import AutoTokenizer
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score



**Select the parameters**

In [ ]:
"""Select the random seed"""
np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

# np.random.seed(12)
# python_random.seed(12)
# tf.random.set_seed(12)

# np.random.seed(123)
# python_random.seed(123)
# tf.random.set_seed(123)

"""Select the language model"""
#lm = "roberta-base"
#lm = "google/electra-base-discriminator"
lm = "google/electra-large-discriminator"

"""Select whether to use the backtranslated data or not"""
#use_backtranslated = True # To use the backtranslated data
use_backtranslated = False # To use the regular (preprocessed) data

In [ ]:
if use_backtranslated:
  df_train = pd.read_csv("merged_train.csv", sep=',', names=['text', 'label'])
  df_dev = pd.read_csv("merged_dev.csv", sep=',', names=['text', 'label'])
  df_test = pd.read_csv("merged_test.csv", sep=',', names=['text', 'label'])
else:
  df_train = pd.read_csv("train.csv", sep=',', names=['text', 'label'])
  df_dev = pd.read_csv("dev.csv", sep=',', names=['text', 'label'])
  df_test = pd.read_csv("test.csv", sep=',', names=['text', 'label'])

# Converting comments to strings
df_train['text'] = df_train['text'].astype(str)
df_dev['text'] = df_dev['text'].astype(str)
df_test['text'] = df_test['text'].astype(str)

df_train.dropna()
df_dev.dropna()
df_test.dropna()

# Loading dataframes into variables
x_train, y_train, x_dev, y_dev, x_test, y_test = df_train['text'], df_train['label'], df_dev['text'], df_dev['label'], df_test['text'], df_test['label']

In [ ]:
# Transform labels into binary
encoder = LabelBinarizer()

Y_train_bin = encoder.fit_transform(y_train)  
Y_dev_bin = encoder.fit_transform(y_dev)
Y_test_bin = encoder.fit_transform(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(lm)
model = TFAutoModelForSequenceClassification.from_pretrained(lm, num_labels=2)
tokens_train = tokenizer(list(x_train), padding=True, max_length=50,truncation=True, return_tensors="np").data
tokens_dev = tokenizer(list(x_dev), padding=True, max_length=50,truncation=True, return_tensors="np").data
tokens_test = tokenizer(list(x_test), padding=True, max_length=50,truncation=True, return_tensors="np").data

Some layers from the model checkpoint at google/electra-large-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
loss_function = SparseCategoricalCrossentropy(from_logits=True)
optim = Adam(learning_rate=5e-6)

model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])

# Fitting the Bert model
model.fit(tokens_train, Y_train_bin, verbose=1, epochs=3,batch_size=32, validation_data=(tokens_dev, Y_dev_bin))
Y_pred = model.predict(tokens_test)#["logits"]

Epoch 1/3
383/383 [==============================] - 449s 1s/step - loss: 0.5497 - accuracy: 0.7284 - val_loss: 0.4658 - val_accuracy: 0.7950
Epoch 2/3
383/383 [==============================] - 404s 1s/step - loss: 0.4283 - accuracy: 0.8114 - val_loss: 0.4377 - val_accuracy: 0.7970
Epoch 3/3
177/383 [============>.................] - ETA: 3:31 - loss: 0.3760 - accuracy: 0.8362

In [ ]:
# Convert to labels to get scores with sklearn
y_pred_labels = np.argmax(Y_pred[0], axis=1)
y_test_labels = Y_test_bin

# # If you have gold data, you can calculate accuracy
f1_score(y_test_labels, y_pred_labels, average='macro')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_labels, y_pred_labels, digits=4))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test_labels, y_pred_labels, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1])
disp.plot()
plt.show()